<a href="https://colab.research.google.com/github/MarinaOhm/master_thesis/blob/main/RICE_FORECASTING_PLOTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master thesis
## *Forecasting plots for Forecasting*
*Developed by Max Hedeman Gueniau, Niklas Madsen, and Marina Ohm*

# Importing libraries

In [ ]:
# Libraries used for data wrangling and visualizations 
import pandas as pd
import numpy as np
import seaborn as sns
import time
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
import plotly.graph_objects as go
from datetime import datetime
from prettytable import PrettyTable
from plotly.subplots import make_subplots


In [ ]:
forecasting_data=pd.read_csv('/content/drive/MyDrive/final_forecasts_rice.csv')#, sep=';')
forecasting_data.head()

,Unnamed: 0,_ItemNumber,Model,Forecast,ConfInt_lower,ConfInt_upper,train_test_RMSE
0,2023-04-16,BGLEA-RVIL,EXP SMOOTH,3.078716,0.115656,6.041776,33.290582
1,2023-04-23,BGLEA-RVIL,EXP SMOOTH,2.183034,-0.780025,5.146094,33.290582
2,2023-04-30,BGLEA-RVIL,EXP SMOOTH,1.610331,-1.352729,4.573391,33.290582
3,2023-05-07,BGLEA-RVIL,EXP SMOOTH,1.227302,-1.735758,4.190361,33.290582
4,2023-05-14,BGLEA-RVIL,EXP SMOOTH,0.961760,-2.001300,3.924819,33.290582


In [ ]:
forecasting_data = forecasting_data.rename(columns={"Unnamed: 0": "date_weekly"})

In [ ]:
forecasting_data.columns

Index(['date_weekly', '_ItemNumber', 'Model', 'Forecast', 'ConfInt_lower',
       'ConfInt_upper', 'train_test_RMSE'],
      dtype='object')

In [ ]:
len(forecasting_data._ItemNumber.unique())

234

In [ ]:
# Read original data to merge with forecasting results, in order to get observed values 
original_data=pd.read_csv('/content/drive/MyDrive/weekly_forecasting data.csv')#, sep=';')
original_data=original_data.drop(['date_'], axis=1)
original_data.head()

,date_weekly,_ItemNumber,Q
0,2016-07-05,BGLEA-RVIL,8.0
1,2016-07-05,BGLEA-RVIL,20.0
2,2016-07-12,BGLEA-RVIL,8.0
3,2016-07-12,BGLEA-RVIL,12.0
4,2016-07-19,BGLEA-RVIL,4.0


In [ ]:
len(original_data._ItemNumber.unique())

234

In [ ]:
# Check that we have all the SKUs 
unique_item_numbers_forecasting = set(forecasting_data["_ItemNumber"].unique())
unique_item_numbers_original = set(original_data["_ItemNumber"].unique())

missing_item_numbers = unique_item_numbers_original.difference(unique_item_numbers_forecasting)

print("Missing item numbers in forecasting_data:")
print(missing_item_numbers)


Missing item numbers in forecasting_data:
set()


In [ ]:
# Add missing columns to both DataFrames with NaN values
#forecasting_data["Q"] = np.nan
forecasting_data["Q"] = np.nan
original_data["Model"] = np.nan
original_data["Forecast"] = np.nan
original_data["ConfInt_lower"] = np.nan
original_data["ConfInt_upper"] = np.nan
original_data["train_test_RMSE"] = np.nan
# Append the DataFrames
result = forecasting_data.append(original_data, ignore_index=True)

#result = forecasting_data_test.append(original_data, ignore_index=True)


<ipython-input-11-cf36a704f5de>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = forecasting_data.append(original_data, ignore_index=True)


In [ ]:
result['Merged'] = result['Q'].fillna(result['Forecast'])

In [ ]:
result["date_weekly"] = pd.to_datetime(result["date_weekly"])

In [ ]:
result["Model"].value_counts()

LSTM          3505
EXP SMOOTH     875
ARIMA          725
XGBOOST        700
Name: Model, dtype: int64

In [ ]:
rmse_stat=forecasting_data[forecasting_data["train_test_RMSE"]<1000]

In [ ]:
rmse_stat["train_test_RMSE"].describe()

count    5555.000000
mean      113.938151
std       145.636210
min         0.136812
25%        16.173032
50%        69.655192
75%       146.521114
max       934.383027
Name: train_test_RMSE, dtype: float64

In [ ]:
len(rmse_stat)

5555

## Color palette

In [ ]:
def assign_colors(unique_values, palette):
    color_mapping = {}
    num_colors = len(palette)

    for idx, value in enumerate(unique_values):
        color_mapping[value] = palette[idx % num_colors]

    return color_mapping

In [ ]:
color_palette = [
    'rgba(41, 57, 71, 1)',
    'rgba(128, 128, 128, 1)',
    'rgba(93, 112, 127, 1)',
    'rgba(155, 170, 181, 1)',
    'rgba(80, 177, 200, 100)','rgba(154, 177, 197, 100)',
    'rgba(133, 173, 219, 100)','rgba(28, 59, 95, 100)',
    'rgba(40, 109, 125, 100)','rgba(23, 63, 73, 100)',
    'rgba(70, 115, 171, 100)'

]

# Evaluation 

## Distribution of RMSE scores

In [ ]:
ranges = [0, 50, 100, 150, 200, 250, 300,  float("inf")]
labels = ['[0-50]', '[51-100]', '[101-150]', '[151-200]','[201-250]','[251-300]','[above 300]']
customer_total_q = forecasting_data.groupby('_ItemNumber')['train_test_RMSE'].mean().reset_index()
customer_total_q['range'] = pd.cut(customer_total_q['train_test_RMSE'], bins=ranges, labels=labels, include_lowest=True)
total_customers_by_range = customer_total_q['range'].value_counts().sort_index()

# Create a DataFrame from the total_customers_by_range_value Series
total_customers_by_range_df = total_customers_by_range.reset_index()
total_customers_by_range_df.columns = ['Range', 'Count']

# Assign colors 
unique_ranges = total_customers_by_range_df['Range'].unique()
range_color_mapping = assign_colors(unique_ranges, color_palette)

fig = px.bar(total_customers_by_range_df, x='Range', y='Count', color='Range', color_discrete_map=range_color_mapping)

fig.update_traces(
    texttemplate='%{y:,.0f}', text=total_customers_by_range.values,
    textposition='outside',
    textfont=dict(size=14, color='black', family='Arial')
)

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(yaxis_title="Count")
fig.update_layout(xaxis_title="Range")
fig.update_layout(title_text="Distribution of RMSE results on all SKUs", title_x=0.5)
fig.show()

## Distribution of best model

In [ ]:
model_count = forecasting_data.groupby('Model')['_ItemNumber'].nunique()
model_count = model_count.sort_values(ascending=False)

# Assign colors 
#color_mapping = assign_colors(model_count, color_palette)
unique_ranges = model_count.index.unique()
color_mapping = assign_colors(unique_ranges, color_palette)

fig = px.bar(x=model_count.index, y=model_count.values,color=model_count.index,color_discrete_map=color_mapping)

values=model_count.values

fig.update_traces(
    texttemplate='%{text:.2s}', text=values,
    textposition='outside',
    textfont=dict(size=14, color='black', family='Arial')
)


fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(yaxis_title="Count")
fig.update_layout(xaxis_title="Model")
fig.update_layout(title_text="Distribution of the best model for each unique SKU", title_x=0.5)
fig.show()

In [ ]:
import plotly.graph_objs as go

model_count = forecasting_data.groupby('Model')['_ItemNumber'].nunique()
model_count = model_count.sort_values(ascending=False)

# Assign colors 
#color_mapping = assign_colors(model_count, color_palette)
unique_ranges = model_count.index.unique()
color_mapping = assign_colors(unique_ranges, color_palette)

fig = go.Figure()

for idx, value in model_count.iteritems():
    fig.add_trace(go.Bar(x=[idx],
                         y=[value],
                         name=idx,
                         marker_color=color_mapping[idx],
                         text=value,
                         textposition='outside',
                         textfont=dict(size=14, color='black', family='Arial')
                         ))

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(yaxis_title="Count")
fig.update_layout(xaxis_title="Model")
fig.update_layout(title_text="Distribution of the best model for each unique SKU", title_x=0.5)
fig.show()


# Model deep dive 

We are going to remove all SKUs with RMSE values above 1,000. The reason for this, is that we found that these models were completely wrong, thus providing a wrong picture of the evaluation. These will be further investigated at a later stage. 

In [ ]:
rmse_stat=forecasting_data[forecasting_data["train_test_RMSE"]<1000]
rmse_stat_above_1000=forecasting_data[forecasting_data["train_test_RMSE"]>1000]
print(len(rmse_stat_above_1000))
print(len(rmse_stat))

250
5555


In [ ]:
rmse_stat_above_1000.Model.value_counts()

ARIMA         100
XGBOOST       100
EXP SMOOTH     50
Name: Model, dtype: int64

In [ ]:
total_sales_week_by_item = result.groupby(['date_weekly', '_ItemNumber'])['Q'].sum()
total_sales_week_by_item.describe()

count    34382.000000
mean       168.600954
std        414.084136
min      -1872.000000
25%          6.000000
50%         43.000000
75%        169.000000
max      17775.000000
Name: Q, dtype: float64

#### LSTM

In [ ]:
lstm=rmse_stat[rmse_stat["Model"]=="LSTM"]
len(lstm)

###### Average RMSE

In [ ]:
lstm.train_test_RMSE.describe()

###### What is the average length of observations?

In [ ]:
unique_items=lstm["_ItemNumber"].unique()

lstm_items = original_data[original_data["_ItemNumber"].isin(unique_items)]
len(lstm_items._ItemNumber.unique())

In [ ]:
observations_by_item_number=pd.pivot_table(lstm_items, index='_ItemNumber', values=['date_weekly'], aggfunc=lambda x: len(x.unique()))

In [ ]:
observations_by_item_number.describe()

In [ ]:
# Sort the pivot table in descending order
sorted_observations = observations_by_item_number.sort_values(by='date_weekly', ascending=False).reset_index()

# Create a bar chart using plotly express
fig = px.bar(sorted_observations, x='_ItemNumber', y='date_weekly', title='Number of Weekly Observations by Unique Item Number',
             color_discrete_sequence=[color_palette[8]])

# Customize the chart layout if needed
fig.update_layout(xaxis_title='Item Number', yaxis_title='Count of Weekly Observations',title_x=0.5)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
# Show the chart
fig.show()


#### EXP SMOOTH

In [ ]:
exp_smooth=rmse_stat[rmse_stat["Model"]=="EXP SMOOTH"]
len(exp_smooth)

###### Average RMSE

In [ ]:
exp_smooth.train_test_RMSE.describe()

###### What is the average length of observations?

In [ ]:
unique_items=exp_smooth["_ItemNumber"].unique()

exp_smooth_items = original_data[original_data["_ItemNumber"].isin(unique_items)]
len(exp_smooth_items._ItemNumber.unique())

In [ ]:
observations_by_item_number=pd.pivot_table(exp_smooth_items, index='_ItemNumber', values=['date_weekly'], aggfunc=lambda x: len(x.unique()))

In [ ]:
observations_by_item_number.describe()

#### ARIMA

In [ ]:
arima=rmse_stat[rmse_stat["Model"]=="ARIMA"]
len(arima)

###### Average RMSE

In [ ]:
arima.train_test_RMSE.describe()

###### What is the average length of observations?

In [ ]:
unique_items=arima["_ItemNumber"].unique()

arima_items = original_data[original_data["_ItemNumber"].isin(unique_items)]
len(arima_items._ItemNumber.unique())

In [ ]:
observations_by_item_number=pd.pivot_table(arima_items, index='_ItemNumber', values=['date_weekly'], aggfunc=lambda x: len(x.unique()))

In [ ]:
observations_by_item_number.describe()

#### XGBOOST

In [ ]:
xgboost=rmse_stat[rmse_stat["Model"]=="XGBOOST"]
len(xgboost)

###### Average RMSE

In [ ]:
xgboost.train_test_RMSE.describe()

###### What is the average length of observations?

In [ ]:
unique_items=xgboost["_ItemNumber"].unique()

xgboost_items = original_data[original_data["_ItemNumber"].isin(unique_items)]
len(xgboost_items._ItemNumber.unique())

In [ ]:
observations_by_item_number=pd.pivot_table(xgboost_items, index='_ItemNumber', values=['date_weekly'], aggfunc=lambda x: len(x.unique()))

In [ ]:
observations_by_item_number.describe()

In [ ]:
len(forecasting_data)

#### Total sum by month 

In [ ]:
# Set the cutoff date
#cutoff_date = pd.to_datetime("2023-03-01")

# Filter the DataFrame based on the "Merged" column values and the cutoff date:
#filtered_df = result[((result["Merged"] < 25000) & (result["date_weekly"] >= cutoff_date)) | (result["date_weekly"] < cutoff_date)]

start_cutoff_date = pd.to_datetime("2023-03-01")
end_cutoff_date = pd.to_datetime("2023-10-01")

# Filter the DataFrame based on the conditions:
filtered_df = result[
    (
        (result["Merged"] < 25000) & 
        (result["date_weekly"] >= start_cutoff_date) & 
        (result["date_weekly"] < end_cutoff_date)
    ) | 
    (result["date_weekly"] < start_cutoff_date)
]

#filtered_df=result[result["Merged"]<10000]
# Create a new 'month' column by extracting the month from the 'date_weekly' column:
filtered_df['month'] = filtered_df['date_weekly'].dt.to_period('M')

# Group by the 'month' column and sum the 'Merged' column:
monthly_sum = filtered_df.groupby('month')['Merged'].sum().reset_index()

monthly_sum

In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Your existing code to create the filtered_df and monthly_sum DataFrames

# Convert the 'month' column back to datetime (first day of the month) for easier plotting
monthly_sum['month'] = monthly_sum['month'].astype(str)
monthly_sum['month'] = pd.to_datetime(monthly_sum['month'])

# Split the data into segments
before_start = monthly_sum[monthly_sum['month'] < start_cutoff_date]
between_start_end = monthly_sum[(monthly_sum['month'] >= start_cutoff_date) & (monthly_sum['month'] < end_cutoff_date)]
after_end = monthly_sum[monthly_sum['month'] >= end_cutoff_date]

# Create the figure and add traces for each segment
trendfig = go.Figure()

# Add trace for data before the start cutoff date
trendfig.add_trace(go.Scatter(x=before_start['month'], y=before_start['Merged'], name='Total Sales', line=dict(color=color_palette[0])))

# Add trace for data between the start and end cutoff dates
trendfig.add_trace(go.Scatter(x=between_start_end['month'], y=between_start_end['Merged'], name='Total Sales (Forecast)', line=dict(color=color_palette[4])))

# Add trace for data after the end cutoff date, if applicable
if not after_end.empty:
    trendfig.add_trace(go.Scatter(x=after_end['month'], y=after_end['Merged'], name='Total Sales (After)', line=dict(color=color_palette[8])))

# Update layout and axes titles
trendfig.update_layout(title='Aggregated forecast compared to observed values', title_x=0.5, xaxis_title='Date', yaxis_title='Sum of quantity (Month)', plot_bgcolor='rgba(0,0,0,0)')

# Show the figure
trendfig.show()


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Master Thesis/FFFF_final_forecasting_data_with2023.csv")
df.head()

In [ ]:
other_df_filtered = df[['_ItemNumber', 'Item Category Code']]
other_df_filtered

In [ ]:
merged_df = result.merge(other_df_filtered, left_on='_ItemNumber', right_on='_ItemNumber', how='left')
merged_df = merged_df.drop('_ItemNumber', axis=1)
merged_df

In [ ]:
import pandas as pd

# Assuming your DataFrames are named 'result' and 'other_df'
# Make sure the column names are correct in your DataFrames

# Select only the 'ItemNumber' and 'Item Category Code' columns from other_df
other_df_filtered = df[['_ItemNumber', 'Item Category Code']]

# Merge DataFrames on the specified columns
merged_df = result.merge(other_df_filtered, left_on='_ItemNumber', right_on='_ItemNumber', how='left')

# Drop the 'ItemNumber' column as it's a duplicate of '_ItemNumber'
merged_df = merged_df.drop('_ItemNumber', axis=1)

# Now 'merged_df' contains the result DataFrame with the 'Item Category Code' column appended
merged_df

In [ ]:
import time
from datetime import datetime

# Set 'date_' column as index
df = df.reset_index()
df["date_"] = pd.to_datetime(df["date_"])
df.set_index('date_', inplace=True)

# Convert the index to a DatetimeIndex
#datetime_index = pd.DatetimeIndex(df.index)

# Create new column 'date_weekly'
df['date_weekly'] = datetime_index - pd.to_timedelta(datetime_index.dayofweek, unit='d') + pd.DateOffset(days=1)
df=df.reset_index()
weekly_df_forecasting = df[["date_weekly", "_ItemNumber", "Q", "Item Category Code"]]
weekly_df_forecasting.head()

#weekly_df_forecasting.to_csv('weekly_forecasting data.csv')

In [ ]:
forecasting_data

In [ ]:

# Create a DataFrame with unique item numbers and their corresponding item category codes
unique_item_categories = weekly_df_forecasting[["_ItemNumber", "Item Category Code"]].drop_duplicates()

# Merge the unique item categories with the forecasting_data DataFrame
forecasting_data = forecasting_data.merge(unique_item_categories, on="_ItemNumber", how="left")

# Set NaN values for the Q column in forecasting_data
#forecasting_data["Q"] = np.nan

# Add new columns with NaN values to the original_data DataFrame
weekly_df_forecasting["Model"] = np.nan
weekly_df_forecasting["Forecast"] = np.nan
weekly_df_forecasting["ConfInt_lower"] = np.nan
weekly_df_forecasting["ConfInt_upper"] = np.nan
weekly_df_forecasting["train_test_RMSE"] = np.nan

# Append the DataFrames
result_with_categories = forecasting_data.append(weekly_df_forecasting, ignore_index=True)
result_with_categories.head()

In [ ]:
result_with_categories['Merged'] = result_with_categories['Q'].fillna(result_with_categories['Forecast'])

In [ ]:
result_with_categories

In [ ]:
start_cutoff_date = pd.to_datetime("2023-03-01")
end_cutoff_date = pd.to_datetime("2023-10-01")

# Filter the DataFrame based on the conditions:
filtered_df = result_with_categories[
    (
        (result_with_categories["Merged"] < 25000) & 
        (result_with_categories["date_weekly"] >= start_cutoff_date) & 
        (result_with_categories["date_weekly"] < end_cutoff_date)
    ) | 
    (result_with_categories["date_weekly"] < start_cutoff_date)
]

#filtered_df=result[result["Merged"]<10000]
# Create a new 'month' column by extracting the month from the 'date_weekly' column:
filtered_df['month'] = filtered_df['date_weekly'].dt.to_period('M')

In [ ]:
import plotly.graph_objs as go

# Convert the 'date_weekly' column to a datetime object
filtered_df['date_weekly'] = pd.to_datetime(filtered_df['date_weekly'])

# Filter the data for years 2020 and onwards
after_2022 = filtered_df[filtered_df['date_weekly'] > pd.to_datetime('2020-01-01')]

# Group the data by Item Category Code and Year
after_2022['year'] = pd.DatetimeIndex(after_2022['date_weekly']).year
category_yearly_sales = after_2022.groupby(['Item Category Code', 'year'], as_index=False)['Merged'].sum()

# Create a color mapping for the unique categories
unique_categories = category_yearly_sales['Item Category Code'].unique()
color_mapping = assign_colors(unique_categories, color_palette)

# Create the bar chart
store_yearly_sales_bars = []
for cat in unique_categories:
    current_store_yearly_sales = category_yearly_sales[(category_yearly_sales['Item Category Code'] == cat)]
    store_yearly_sales_bars.append(go.Bar(x=current_store_yearly_sales['year'], y=current_store_yearly_sales['Merged'], name=('Category: %s' % cat), marker=dict(color=color_mapping[cat])))

layout = go.Layout(title='Yearly quantity sold by category from 2020 onwards', title_x=0.5, xaxis=dict(title=''), yaxis=dict(title='Quantity'))
fig = go.Figure(data=store_yearly_sales_bars, layout=layout)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5), legend_title="")

fig.show()


In [ ]:

# Convert the 'date_weekly' column to a datetime object
filtered_df['date_weekly'] = pd.to_datetime(filtered_df['date_weekly'])

# Filter the data for years 2020 and onwards
after_2022 = filtered_df[filtered_df['date_weekly'] > pd.to_datetime('2023-03-01')]

# Group the data by Item Category Code and Year
#after_2022['month'] = pd.DatetimeIndex(after_2022['date_weekly']).year
category_yearly_sales = after_2022.groupby(['Item Category Code', 'month'], as_index=False)['Merged'].sum()
category_yearly_sales['month'] = category_yearly_sales['month'].astype(str)

# Create a color mapping for the unique categories
unique_categories = category_yearly_sales['Item Category Code'].unique()
color_mapping = assign_colors(unique_categories, color_palette)

# Create the bar chart
store_yearly_sales_bars = []
for cat in unique_categories:
    current_store_yearly_sales = category_yearly_sales[(category_yearly_sales['Item Category Code'] == cat)]
    store_yearly_sales_bars.append(go.Bar(x=current_store_yearly_sales['month'], y=current_store_yearly_sales['Merged'], name=('Category: %s' % cat), marker=dict(color=color_mapping[cat])))

layout = go.Layout(title='Monthly forecast by category', title_x=0.5, xaxis=dict(title=''), yaxis=dict(title='Quantity'))
fig = go.Figure(data=store_yearly_sales_bars, layout=layout)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=-0.2, xanchor='center', x=0.5), legend_title="")

fig.show()

## Forecast plots

In [ ]:

# Sort the DataFrame by 'train_test_RMSE'
sorted_results = result.sort_values(by='train_test_RMSE')

# Drop duplicates to keep only the lowest RMSE row for each _ItemNumber
unique_sorted_results = sorted_results.drop_duplicates(subset='_ItemNumber')

# Select the top 20 _ItemNumber with the lowest train_test_RMSE
top_20_items = unique_sorted_results.head(20)

# Reset the index of the new DataFrame
top_20_items.reset_index(drop=True, inplace=True)


In [ ]:
long_observations_lstm=sorted_observations.head(30)
long_observations_lstm

In [ ]:

# Choose a random _ItemNumber
#item_number_values = ['MELCU-II', 'MELCU-DMI', 'MELCU-LMAGT', 'WRBIR-SXC']
item_number_values = top_20_items["_ItemNumber"].unique()
#unique_item_numbers = top_10_lowest_rmse["_ItemNumber"].unique()
item_number = np.random.choice(item_number_values, size=1, replace=False)[0]

# Filter the data for the chosen _ItemNumber and sort by date_weekly
filtered_data = result[result["_ItemNumber"] == item_number].sort_values(by="date_weekly")
#filtered_data=result[result["train_test_RMSE"]<1000]
# Create a new 'month' column by extracting the month from the 'date_weekly' column:

model_name = filtered_data["Model"].dropna().iloc[0]
total_sales_week = filtered_data.groupby('date_weekly').sum()['Merged']
total_sales_week = total_sales_week.reset_index()
#total_sales_week = total_sales_week[total_sales_week["Merged"] > 0]
#total_sales_week = total_sales_week[total_sales_week["date_weekly"] > '2022-01-01']
#filtered_data = filtered_data[filtered_data["date_weekly"] > '2022-01-01']

n_forecast = 25
forecast_start_idx = len(total_sales_week) - n_forecast

# Create the plot
fig = go.Figure()

# Plot the observed data
fig.add_trace(go.Scatter(x=total_sales_week["date_weekly"][:forecast_start_idx], y=total_sales_week["Merged"][:forecast_start_idx],
                         mode='lines', name='Observed', line=dict(color=color_palette[0]), marker=dict(color='blue')))

# Plot the forecasted data
fig.add_trace(go.Scatter(x=total_sales_week["date_weekly"][forecast_start_idx:], y=total_sales_week["Merged"][forecast_start_idx:],
                         mode='lines', name='Forecast', line=dict(color=color_palette[4],dash="dash"), marker=dict(color='red')))

# Set the plot title and labels
#fig.update_layout(title=f"Forecast for _ItemNumber: {item_number}", title_x=0.5,
 #                 xaxis_title="date_weekly",
  #                yaxis_title="Values", plot_bgcolor='rgba(0,0,0,0)')

# Set the plot title and labels
fig.update_layout(title=f"Forecast for Item Number: {item_number} using {model_name}",title_x=0.5,
                  xaxis_title="date_weekly",
                  yaxis_title="Values", plot_bgcolor='rgba(0,0,0,0)')

# Display the plot
fig.show()


<ipython-input-41-5a651ae39f12>:17: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [ ]:
import plotly.graph_objects as go

# Choose a random _ItemNumber
#item_number = "CDCAK-TWISTG"
#item_number_values = ['MELCU-II','MELCU-DMI','MELCU-LMAGT','WRBIR-SXC']
#item_number_values = ['MELCU-II','MELCU-DMI','MELCU-LMAGT','WRBIR-SXC']
unique_item_numbers = result["_ItemNumber"].unique()
unique_item_numbers = top_20_models["_ItemNumber"].unique()
item_number = np.random.choice(unique_item_numbers, size=1, replace=False)[0]

# Filter the data for the chosen _ItemNumber and sort by date_weekly
filtered_data = result[result["_ItemNumber"] == item_number].sort_values(by="date_weekly")

model_name = filtered_data["Model"].dropna().iloc[0]

total_sales_week = filtered_data.groupby('date_weekly').sum()['Q']
total_sales_week = total_sales_week.reset_index()
total_sales_week = total_sales_week[total_sales_week["Q"] > 0]
#total_sales_week = total_sales_week[total_sales_week["date_weekly"] > '2020-01-01']
filtered_data = filtered_data[filtered_data["date_weekly"] > '2022-01-01']

# Create the plot
fig = go.Figure()

# Plot the observed data
fig.add_trace(go.Scatter(x=total_sales_week["date_weekly"], y=total_sales_week["Q"],
                         mode='lines', name='Observed', line=dict(color=color_palette[0])))

# Plot the forecasted data
fig.add_trace(go.Scatter(x=filtered_data["date_weekly"], y=filtered_data["Forecast"],
                         mode='lines', name=f'Forecast ({model_name})', line=dict(color=color_palette[4],dash="dash")))

# Plot the confidence intervals
fig.add_trace(go.Scatter(x=filtered_data["date_weekly"], y=filtered_data["ConfInt_upper"],
                         mode='lines', name='Upper Confidence Interval', line=dict(color='rgba(154, 177, 197, 0.2)'), showlegend=False))

fig.add_trace(go.Scatter(x=filtered_data["date_weekly"], y=filtered_data["ConfInt_lower"],
                         mode='lines', name='Lower Confidence Interval', line=dict(color='rgba(154, 177, 197, 0.2)'), showlegend=False, fill='tonexty'))


# Set the plot title and labels
fig.update_layout(title=f"Forecast for Item Number: {item_number} using {model_name}",title_x=0.5,
                  xaxis_title="date_weekly",
                  yaxis_title="Values", plot_bgcolor='rgba(0,0,0,0)')


# Display the plot
fig.show()
